# 光子の干渉データ解析

数値計算`numpy`、データ解析`pandas`パッケージの読み込み。  
`C++/C`の経験者であれば、`import`は`#include`に対応すると理解してください。

In [ ]:
import numpy as np
import pandas as pd

### データファイル (excel) の読み込み
`pandas.read_excel()`関数はエクセルファイルを`pandas`のデータフレームというコンテナ(以下では`df`としている)にロードする。  
以下ではコンテナ`df`にアクセス、処理することで表計算処理が可能となる。  
`photo_if_data.xlsx`は過去の実験データ。  
データフォーマットは　
- 1列目 光電子増倍管の位置 (x)
- ２列目 レーザー強度 (V)
- 3列目 カウント数 (N)
- 4列目 N/V
としている。  
各自、ファイルにおけるデータ列の並びを考慮して以下のコードを書き換える。  
なお、引数で用いられている`sheet_name`はエクセルのシートの名前, `usecols`は読み込む列番号 (ここでは0,1,2,3としている),  
`names`で各列の名前を定義している。  
他にも`read_excel()`関数には様々なオプションが用意されている。  

<div class="alert alert-block alert-info">
<b>参考 csvファイルの読み込み</b><br>
pandas.read_csv() 関数はcsvファイルをpandasのデータフレームというコンテナ(以下ではdfとしている)にロードする。<br>
以下dfにアクセスを処理することで表計算処理が可能となる。 <br>
例として、データフォーマットは一列目光電子増倍管の位置、２列目　光電子増倍管のカウント数/レーザー強度。<br>
photo_if_data.csvは過去の実験データ。  
</div>


In [ ]:
df = pd.read_excel('photo_if_data.xlsx', sheet_name='Sheet2',usecols=[0,1,2,3],names=('X', 'V','N','NV'))
# df = pd.read_csv('photo_if_data.csv',names=('X','NV'))

最初の５行を見てみる。

In [ ]:
df.head()

最後の５行を見るには？

### 描画パッケージmatplotlibの読み込み
パッケージ名が長いので`plt`と名前をつけるのが通例。

In [ ]:
import matplotlib.pyplot as plt

### グラフ作成

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
_ = ax.plot(df.X,df.NV,'bo-')

In [ ]:
df['V'].hist(bins=50)

### 理論カーブの考察

レーザースポットの広がりを考慮すると、ガウス分布関数
$$
G(x) = Ae^{-\frac{1}{2}\frac{(x-m)^2}{s^2}}
$$
干渉パターン
$$
I(x) = (1+cos(ksin\theta x)) = (1+cos((2\pi/d)x)
$$
実験データはおそらくこの２つの関数の掛け算で表現できると予想される。  
実際に２つの関数をかけ合わせたグラフを作成する。

In [ ]:
x = np.linspace(0,10,100)
y = 50000*np.exp(-0.5*(x-3.5)**2/2**2)*(1+np.cos(((2*np.pi*(x+0.4))/1)))
# y = 50000*np.exp(-0.5*(x-4)**2/2**2)*(1+np.cos(((2*np.pi*(x+0.43))/1)))

In [ ]:
fig = plt.figure(figsize=(6,4))
ax = fig.add_subplot(111)
_ = ax.plot(df.X,df.NV,'o-')
_ = ax.plot(x,y,'-')

### フィッティング関数の読み込み
上記のように、おおよそのパラーメータの値を推測してデータを再現してもよいが  
正しくは最適なパラメータをフィッティングで求めることとなる。  
フィッティングには`scipy.optimize.curve_fit()`関数を用いることができる。

In [ ]:
from scipy.optimize import curve_fit

使い方は、簡単に言うと
```python
curve_fit(関数, データx, データy, 初期値)
```
とすればよい。  
返り値として最適化されたパラメータのリスト （+covariant matrix）。


In [ ]:
help(curve_fit)

### フィッティング関数の定義
`fitFun()`の`a,b,c,d,e`がパラメータ。

In [ ]:
def fitFun(x,a,b,c,d,e):
    return a*np.exp(-0.5*(x-b)**2/c**2)*(1+np.cos((2*np.pi*(x+e)/d)))

In [ ]:
par,cov = curve_fit(fitFun,df.X,df.NV,p0=(200000,4.5,2,1,0.5))

In [ ]:
par

結果を表示してみる。

In [ ]:
fig = plt.figure(figsize=(6,4))
y = fitFun(x,par[0],par[1],par[2],par[3],par[4])
ax = fig.add_subplot(111)
_ = ax.plot(df.X,df.NV,'o-')
_ = ax.plot(x,y,'-')
plt.savefig('pif.png') # グラフをpngファイルとして保存する関数。